In [66]:
import pandas as pd
from nltk import word_tokenize
import nltk
from nltk.util import ngrams
from collections import Counter
import numpy as np
import spacy
import en_core_web_lg
pd.set_option('display.max_columns', 50)

ModuleNotFoundError: No module named 'en_core_web_lg'

In [3]:
df = context.io.load('KOF_04152019_04')

2019-12-07 23:12:25,193 - kedro.io.data_catalog - INFO - Loading data from `KOF_04152019_04` (CSVLocalDataSet)...


In [4]:
df = df.drop(columns='Unnamed: 0')

In [5]:
df = df.rename(columns = {'Code_Style':'code_style', 'Name':'name', 'Brand':'brand', 'Date':'date',
                    'Retail_Price': 'retail_price', 'Colorway':'colorway', 'Story':'story',
                    'KOF_Wants':'kof_wants', 'Avg_Resale':'avg_resale_stockx'})

In [6]:
df['retail_price'] = df['retail_price'].astype(int)
df['avg_resale_stockx'] = df['avg_resale_stockx'].str.replace('[^\w\s]','')
df['avg_resale_stockx'] = df['avg_resale_stockx'].astype(int)

In [7]:
df.head(5)

,code_style,name,brand,date,retail_price,colorway,story,kof_wants,avg_resale_stockx
0,314996-101,Nike Air Foamposite One Snakeskin,Nike,2019-04-15,220,Sail/Black-Habanero Red-Black,"Equipped with an eye-catching upper, this Air ...",2281,305
1,BV4580-400,Undercover x Nike SFB Mountain Obsidian Univer...,Nike,2019-04-15,350,Obsidian/University Red-Dark Obsidian,The Undercover x Nike SFB Mountain Obsidian/Un...,37,192
2,BV4580-001,Undercover x Nike SFB Mountain Black,Nike,2019-04-15,350,Black/Sail-Black,The Undercover x Nike SFB Mountain Black is In...,70,231
3,AO2607-900,Nike PG 3 Mamba Mentality,Nike,2019-04-13,110,Multi-Color/Opti Yellow,The Nike PG 3 Mamba Mentality comes covered in...,179,109
4,AO2918-102,Nike Kyrie 5 Mamba Mentality,Nike,2019-04-13,130,White/Cyber,The Nike Kyrie 5 Mamba Mentality is a new rend...,271,107


## Create "purchase" feature


In [8]:
#create columns to calculate net profit
df['price_diff'] = df['avg_resale_stockx'] - df['retail_price']
df['commission_fee'] = abs((df['avg_resale_stockx']) * (9.5/100))
df['seller_fee'] = 5
df['total_credit'] = df['price_diff'] - df['commission_fee'] - df['seller_fee']
df['cashout_fee'] = abs((df['total_credit']) * (2.9/100))
df['net_profit'] = df['total_credit'] - df['cashout_fee']

2019-12-07 23:12:31,159 - numexpr.utils - INFO - NumExpr defaulting to 4 threads.


In [9]:
#create purchase feature if the net profit is greater than 0
df['purchase'] = np.where(df['net_profit'] > 0, 1, 0)

In [10]:
#drop columns used to calculate net profit
df.drop(['commission_fee', 'seller_fee', 'total_credit', 'cashout_fee'], axis=1, inplace=True)

In [11]:
df.head(1)

,code_style,name,brand,date,retail_price,colorway,story,kof_wants,avg_resale_stockx,price_diff,net_profit,purchase
0,314996-101,Nike Air Foamposite One Snakeskin,Nike,2019-04-15,220,Sail/Black-Habanero Red-Black,"Equipped with an eye-catching upper, this Air ...",2281,305,85,49.545275,1


## Create "brand_code" feature

In [12]:
brand_code = df.groupby('brand').ngroup()
df = pd.concat([df, brand_code], axis=1).rename(columns={0:'brand_code'})


In [13]:
df.head(1)

,code_style,name,brand,date,retail_price,colorway,story,kof_wants,avg_resale_stockx,price_diff,net_profit,purchase,brand_code
0,314996-101,Nike Air Foamposite One Snakeskin,Nike,2019-04-15,220,Sail/Black-Habanero Red-Black,"Equipped with an eye-catching upper, this Air ...",2281,305,85,49.545275,1,6


## Create "Color_Code" feature

## Create "Womens" feature

In [14]:
def label_womens(x):
    name = x
    name = name.lower()
    
    checker1 = 'wmns' in name
    checker2 = 'womens' in name
    checker3 = "women's" in name

    if checker1 == True or checker2 == True or checker3 == True:
        return 1
    else:
        return 0

In [15]:
df['womens'] = df['name'].apply(lambda x: label_womens(x))

In [17]:
df.head(1)

,code_style,name,brand,date,retail_price,colorway,story,kof_wants,avg_resale_stockx,price_diff,net_profit,purchase,brand_code,womens
0,314996-101,Nike Air Foamposite One Snakeskin,Nike,2019-04-15,220,Sail/Black-Habanero Red-Black,"Equipped with an eye-catching upper, this Air ...",2281,305,85,49.545275,1,6,0


## Create "bcollab" feature

In [18]:
def label_bcollab(x):
    name = x
    name = name.lower()
    
    checker1 = ' x ' in name

    if checker1 == True:
        return 1
    else:
        return 0

In [19]:
df['bcollab'] = df['name'].apply(lambda x: label_bcollab(x))

In [21]:
df.head(1)

,code_style,name,brand,date,retail_price,colorway,story,kof_wants,avg_resale_stockx,price_diff,net_profit,purchase,brand_code,womens,bcollab
0,314996-101,Nike Air Foamposite One Snakeskin,Nike,2019-04-15,220,Sail/Black-Habanero Red-Black,"Equipped with an eye-catching upper, this Air ...",2281,305,85,49.545275,1,6,0,0


## Create "OG" feature

In [22]:
def label_og(x):
    name = x
    name = name.lower()
    
    checker1 = ' og' in name

    if checker1 == True:
        return 1
    else:
        return 0

In [23]:
df['og'] = df['name'].apply(lambda x: label_og(x))

In [26]:
df.head(1)

,code_style,name,brand,date,retail_price,colorway,story,kof_wants,avg_resale_stockx,price_diff,net_profit,purchase,brand_code,womens,bcollab,og
0,314996-101,Nike Air Foamposite One Snakeskin,Nike,2019-04-15,220,Sail/Black-Habanero Red-Black,"Equipped with an eye-catching upper, this Air ...",2281,305,85,49.545275,1,6,0,0,0


## Create "SP" feature

Nike SP represents the highest level of Nike quality, and also an avenue for Nike’s creative endeavors. All NikeLab releases are SP and some collabs are also categorized as SP.

Example: Nike Free Flyknit Mercurial SP

https://straatosphere.com/straatopedia-sneaker-terminology-guide/

In [27]:
def label_sp(x):
    name = x
    name = name.lower()
    
    checker1 = ' sp' in name

    if checker1 == True:
        return 1
    else:
        return 0

In [28]:
df['sp'] = df['name'].apply(lambda x: label_sp(x))

In [29]:
df.head(1)

,code_style,name,brand,date,retail_price,colorway,story,kof_wants,avg_resale_stockx,price_diff,net_profit,purchase,brand_code,womens,bcollab,og,sp
0,314996-101,Nike Air Foamposite One Snakeskin,Nike,2019-04-15,220,Sail/Black-Habanero Red-Black,"Equipped with an eye-catching upper, this Air ...",2281,305,85,49.545275,1,6,0,0,0,0


## Create "QS" feature

Nike SP represents the highest level of Nike quality, and also an avenue for Nike’s creative endeavors. All NikeLab releases are SP and some collabs are also categorized as SP.

Example: Nike Free Flyknit Mercurial SP

https://straatosphere.com/straatopedia-sneaker-terminology-guide/

In [30]:
def label_qs(x):
    name = x
    name = name.lower()
    
    checker1 = ' qs' in name

    if checker1 == True:
        return 1
    else:
        return 0

In [31]:
df['qs'] = df['name'].apply(lambda x: label_qs(x))

In [32]:
df.head(1)

,code_style,name,brand,date,retail_price,colorway,story,kof_wants,avg_resale_stockx,price_diff,net_profit,purchase,brand_code,womens,bcollab,og,sp,qs
0,314996-101,Nike Air Foamposite One Snakeskin,Nike,2019-04-15,220,Sail/Black-Habanero Red-Black,"Equipped with an eye-catching upper, this Air ...",2281,305,85,49.545275,1,6,0,0,0,0,0


## Create "SB" feature

Nike SP represents the highest level of Nike quality, and also an avenue for Nike’s creative endeavors. All NikeLab releases are SP and some collabs are also categorized as SP.

Example: Nike Free Flyknit Mercurial SP

https://straatosphere.com/straatopedia-sneaker-terminology-guide/

In [33]:
def label_sb(x):
    name = x
    name = name.lower()
    
    checker1 = ' sb' in name

    if checker1 == True:
        return 1
    else:
        return 0

In [34]:
df['sb'] = df['name'].apply(lambda x: label_sb(x))

In [35]:
df.head(1)

,code_style,name,brand,date,retail_price,colorway,story,kof_wants,avg_resale_stockx,price_diff,net_profit,purchase,brand_code,womens,bcollab,og,sp,qs,sb
0,314996-101,Nike Air Foamposite One Snakeskin,Nike,2019-04-15,220,Sail/Black-Habanero Red-Black,"Equipped with an eye-catching upper, this Air ...",2281,305,85,49.545275,1,6,0,0,0,0,0,0


## Create "LS" feature

Nike SP represents the highest level of Nike quality, and also an avenue for Nike’s creative endeavors. All NikeLab releases are SP and some collabs are also categorized as SP.

Example: Nike Free Flyknit Mercurial SP

https://straatosphere.com/straatopedia-sneaker-terminology-guide/

In [36]:
def label_ls(x):
    name = x
    name = name.lower()
    
    checker1 = ' ls' in name

    if checker1 == True:
        return 1
    else:
        return 0

In [37]:
df['ls'] = df['name'].apply(lambda x: label_ls(x))

In [38]:
df.head(1)

,code_style,name,brand,date,retail_price,colorway,story,kof_wants,avg_resale_stockx,price_diff,net_profit,purchase,brand_code,womens,bcollab,og,sp,qs,sb,ls
0,314996-101,Nike Air Foamposite One Snakeskin,Nike,2019-04-15,220,Sail/Black-Habanero Red-Black,"Equipped with an eye-catching upper, this Air ...",2281,305,85,49.545275,1,6,0,0,0,0,0,0,0


## Create "NRG" feature

Nike SP represents the highest level of Nike quality, and also an avenue for Nike’s creative endeavors. All NikeLab releases are SP and some collabs are also categorized as SP.

Example: Nike Free Flyknit Mercurial SP

https://straatosphere.com/straatopedia-sneaker-terminology-guide/

In [39]:
def label_nrg(x):
    name = x
    name = name.lower()
    
    checker1 = ' nrg' in name

    if checker1 == True:
        return 1
    else:
        return 0

In [40]:
df['nrg'] = df['name'].apply(lambda x: label_nrg(x))

In [41]:
df.head(1)

,code_style,name,brand,date,retail_price,colorway,story,kof_wants,avg_resale_stockx,price_diff,net_profit,purchase,brand_code,womens,bcollab,og,sp,qs,sb,ls,nrg
0,314996-101,Nike Air Foamposite One Snakeskin,Nike,2019-04-15,220,Sail/Black-Habanero Red-Black,"Equipped with an eye-catching upper, this Air ...",2281,305,85,49.545275,1,6,0,0,0,0,0,0,0,0


## Create "PRM" feature

Nike SP represents the highest level of Nike quality, and also an avenue for Nike’s creative endeavors. All NikeLab releases are SP and some collabs are also categorized as SP.

Example: Nike Free Flyknit Mercurial SP

https://straatosphere.com/straatopedia-sneaker-terminology-guide/

In [42]:
def label_prm(x):
    name = x
    name = name.lower()
    
    checker1 = ' prm' in name

    if checker1 == True:
        return 1
    else:
        return 0

In [43]:
df['prm'] = df['name'].apply(lambda x: label_prm(x))

In [44]:
df.head(1)

,code_style,name,brand,date,retail_price,colorway,story,kof_wants,avg_resale_stockx,price_diff,net_profit,purchase,brand_code,womens,bcollab,og,sp,qs,sb,ls,nrg,prm
0,314996-101,Nike Air Foamposite One Snakeskin,Nike,2019-04-15,220,Sail/Black-Habanero Red-Black,"Equipped with an eye-catching upper, this Air ...",2281,305,85,49.545275,1,6,0,0,0,0,0,0,0,0,0


## Create "NSW" feature

Nike SP represents the highest level of Nike quality, and also an avenue for Nike’s creative endeavors. All NikeLab releases are SP and some collabs are also categorized as SP.

Example: Nike Free Flyknit Mercurial SP

https://straatosphere.com/straatopedia-sneaker-terminology-guide/

In [45]:
def label_nsw(x):
    name = x
    name = name.lower()
    
    checker1 = ' nsw' in name

    if checker1 == True:
        return 1
    else:
        return 0

In [46]:
df['nsw'] = df['name'].apply(lambda x: label_nsw(x))

In [47]:
df.head(1)

,code_style,name,brand,date,retail_price,colorway,story,kof_wants,avg_resale_stockx,price_diff,net_profit,purchase,brand_code,womens,bcollab,og,sp,qs,sb,ls,nrg,prm,nsw
0,314996-101,Nike Air Foamposite One Snakeskin,Nike,2019-04-15,220,Sail/Black-Habanero Red-Black,"Equipped with an eye-catching upper, this Air ...",2281,305,85,49.545275,1,6,0,0,0,0,0,0,0,0,0,0


## Create "RETRO" feature

Nike SP represents the highest level of Nike quality, and also an avenue for Nike’s creative endeavors. All NikeLab releases are SP and some collabs are also categorized as SP.

Example: Nike Free Flyknit Mercurial SP

https://straatosphere.com/straatopedia-sneaker-terminology-guide/

In [48]:
def label_retro(x):
    name = x
    name = name.lower()
    
    checker1 = ' retro' in name

    if checker1 == True:
        return 1
    else:
        return 0

In [49]:
df['retro'] = df['name'].apply(lambda x: label_retro(x))

In [50]:
df.head(1)

,code_style,name,brand,date,retail_price,colorway,story,kof_wants,avg_resale_stockx,price_diff,net_profit,purchase,brand_code,womens,bcollab,og,sp,qs,sb,ls,nrg,prm,nsw,retro
0,314996-101,Nike Air Foamposite One Snakeskin,Nike,2019-04-15,220,Sail/Black-Habanero Red-Black,"Equipped with an eye-catching upper, this Air ...",2281,305,85,49.545275,1,6,0,0,0,0,0,0,0,0,0,0,0


## Create "SE" feature

Nike SP represents the highest level of Nike quality, and also an avenue for Nike’s creative endeavors. All NikeLab releases are SP and some collabs are also categorized as SP.

Example: Nike Free Flyknit Mercurial SP

https://straatosphere.com/straatopedia-sneaker-terminology-guide/

In [51]:
def label_se(x):
    name = x
    name = name.lower()
    
    checker1 = ' se' in name

    if checker1 == True:
        return 1
    else:
        return 0

In [52]:
df['se'] = df['name'].apply(lambda x: label_se(x))

In [53]:
df.head(1)

,code_style,name,brand,date,retail_price,colorway,story,kof_wants,avg_resale_stockx,price_diff,net_profit,purchase,brand_code,womens,bcollab,og,sp,qs,sb,ls,nrg,prm,nsw,retro,se
0,314996-101,Nike Air Foamposite One Snakeskin,Nike,2019-04-15,220,Sail/Black-Habanero Red-Black,"Equipped with an eye-catching upper, this Air ...",2281,305,85,49.545275,1,6,0,0,0,0,0,0,0,0,0,0,0,0


## Create "PE" feature

Nike SP represents the highest level of Nike quality, and also an avenue for Nike’s creative endeavors. All NikeLab releases are SP and some collabs are also categorized as SP.

Example: Nike Free Flyknit Mercurial SP

https://straatosphere.com/straatopedia-sneaker-terminology-guide/

In [56]:
def label_pe(x):
    name = x
    name = name.lower()
    
    checker1 = ' pe' in name

    if checker1 == True:
        return 1
    else:
        return 0

In [57]:
df['pe'] = df['name'].apply(lambda x: label_pe(x))

In [58]:
df.head(1)

,code_style,name,brand,date,retail_price,colorway,story,kof_wants,avg_resale_stockx,price_diff,net_profit,purchase,brand_code,womens,bcollab,og,sp,qs,sb,ls,nrg,prm,nsw,retro,se,pe
0,314996-101,Nike Air Foamposite One Snakeskin,Nike,2019-04-15,220,Sail/Black-Habanero Red-Black,"Equipped with an eye-catching upper, this Air ...",2281,305,85,49.545275,1,6,0,0,0,0,0,0,0,0,0,0,0,0,0


## Create "GS" feature

Nike SP represents the highest level of Nike quality, and also an avenue for Nike’s creative endeavors. All NikeLab releases are SP and some collabs are also categorized as SP.

Example: Nike Free Flyknit Mercurial SP

https://straatosphere.com/straatopedia-sneaker-terminology-guide/

In [59]:
def label_gs(x):
    name = x
    name = name.lower()
    
    checker1 = ' gs' in name

    if checker1 == True:
        return 1
    else:
        return 0

In [60]:
df['gs'] = df['name'].apply(lambda x: label_gs(x))

In [61]:
df.head(1)

,code_style,name,brand,date,retail_price,colorway,story,kof_wants,avg_resale_stockx,price_diff,net_profit,purchase,brand_code,womens,bcollab,og,sp,qs,sb,ls,nrg,prm,nsw,retro,se,pe,gs
0,314996-101,Nike Air Foamposite One Snakeskin,Nike,2019-04-15,220,Sail/Black-Habanero Red-Black,"Equipped with an eye-catching upper, this Air ...",2281,305,85,49.545275,1,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Create "HS" feature

HS – Hyperstrike Shoes labeled HS are the most exclusive of all. They come in very limited quantities, with the majority given to friends and families (FNF) of artists and celebrities. 

Example: Nike Air Force 1 “Playstation”, which was a promotional shoe given to Sony employees back in 2006.

https://straatosphere.com/straatopedia-sneaker-terminology-guide/

### None were found in this data set. Was skipped for this iteration of the DB.

In [62]:
def label_hs(x):
    name = x
    name = name.lower()
    
    checker1 = ' hs' in name

    if checker1 == True:
        return 1
    else:
        return 0

In [63]:
df['hs'] = df['name'].apply(lambda x: label_hs(x))

In [64]:
df.head(1)

,code_style,name,brand,date,retail_price,colorway,story,kof_wants,avg_resale_stockx,price_diff,net_profit,purchase,brand_code,womens,bcollab,og,sp,qs,sb,ls,nrg,prm,nsw,retro,se,pe,gs,hs
0,314996-101,Nike Air Foamposite One Snakeskin,Nike,2019-04-15,220,Sail/Black-Habanero Red-Black,"Equipped with an eye-catching upper, this Air ...",2281,305,85,49.545275,1,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## FINAL_DF

In [ ]:
df

In [ ]:
df.to_csv('KOF_04152019-modelv2.csv', index=False)